In [32]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import torch_pruning as tp
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import cv2
from tqdm import tqdm
from torchsummary import summary
from fvcore.nn import FlopCountAnalysis, parameter_count
from ptflops import get_model_complexity_info
import time
from torch.fx import symbolic_trace

In [33]:
# Device 설정
device = torch.device("cpu")
print(f"Using device: {device}")

Using device: cpu


In [34]:
# 데이터셋 경로 설정
data_dir = "data"
# 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

In [35]:
# 데이터셋 로드
dataset = ImageFolder(root=data_dir, transform=transform)
print(f"Classes: {dataset.classes}")

Classes: ['with_mask', 'without_mask']


In [36]:
# Train:Val:Test = 70:15:15 분할
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

In [37]:
# DataLoader 생성
batch_size = 16
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [38]:
class MaskClassifier(nn.Module):
    def __init__(self):
        super(MaskClassifier, self).__init__()
        
        # Feature Extraction - 더 얕은 구조로 변경
        self.features = nn.Sequential(
            # First Block
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.2),
            
            # Second Block
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.2),
            
            # Third Block
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.2),
        )
        
        # Classifier - 더 단순한 구조로 변경
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [39]:
def apply_pruning(model, amount=0.5):
    # Initialize pruner
    example_inputs = torch.randn(1, 3, 224, 224)
    imp = tp.importance.MagnitudeImportance()
    
    pruner = tp.pruner.MagnitudePruner(
        model,
        example_inputs,
        importance=imp,
        ch_sparsity=amount,
        ignored_layers=[model.classifier[2]]  # Ignore final classification layer
    )
    
    # Create pruning plan
    pruner.step()
    return model

In [40]:
# Loss function과 optimizer 설정
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001

In [41]:
def train_model(model, train_loader, val_loader, epochs=10):
    best_val_acc = 0.0
    optimizer = optim.Adam(
    model.parameters(),
    lr=learning_rate,
    betas=(0.9, 0.999),
    weight_decay=0.0001
    )
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        correct = 0
        total = 0
        
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}", unit="batch")
        
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            # Progress Bar에 현재 배치의 accuracy 표시
            batch_acc = 100. * correct / total
            progress_bar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{batch_acc:.2f}%'
            })
    
    # 최종 학습 결과 평가
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    final_acc = 100. * correct / total
    print("\n=== Final Training Results ===")
    print(f"Final Validation Accuracy: {final_acc:.2f}%")

In [42]:
# 테스트 함수
def test_model(model, test_loader):
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        progress_bar = tqdm(test_loader, desc="Testing", unit="batch")  # Progress Bar 추가
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            
            progress_bar.set_postfix(batch_accuracy=(preds == labels).float().mean().item())
            
    print("\nTest Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=dataset.classes))

In [43]:
def get_model_size(model):
    """Calculate model size and other metrics in detail"""
    # 파라미터 수 계산
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    
    # 레이어별 파라미터 수 계산
    layer_params = {}
    for name, param in model.named_parameters():
        layer_params[name] = param.numel()
    
    # 메모리 사용량 계산
    param_size = 0
    buffer_size = 0
    
    # 파라미터 메모리 계산
    for param in model.state_dict().values():
        param_size += param.numel() * param.element_size()
    
    # 버퍼 메모리 계산 (BatchNorm 등의 running stats)
    for buffer in model.buffers():
        buffer_size += buffer.numel() * buffer.element_size()
    
    total_size = param_size + buffer_size
    
    # 모델 구조 분석
    layer_types = {}
    for name, module in model.named_modules():
        layer_type = module.__class__.__name__
        if layer_type not in layer_types:
            layer_types[layer_type] = 1
        else:
            layer_types[layer_type] += 1
    
    return {
        'model_stats': {
            'trainable_parameters': trainable_params,
            'total_parameters': total_params,
            'param_memory_bytes': param_size,
            'buffer_memory_bytes': buffer_size,
            'total_memory_bytes': total_size,
            'total_memory_mb': total_size / (1024 * 1024)
        },
        'layer_parameters': layer_params,
        'layer_types': layer_types
    }

In [44]:
def create_and_train_pruned_model(pruning_ratio, train_loader, val_loader, num_epochs=20):
    """Create, prune, and train a model with specified pruning ratio"""
    # 새 모델 생성 및 기존 가중치 로드
    model = MaskClassifier().to(device)
    model.load_state_dict(torch.load("mask_classifier.pth"))
    
    # Pruning 적용
    print(f"\nApplying {pruning_ratio*100}% pruning...")
    apply_pruning(model, amount=pruning_ratio)
    
    # 모델 학습
    print(f"\nTraining {pruning_ratio*100}% pruned model...")
    train_model(model, train_loader, val_loader, num_epochs)
    
    # 모델 저장
    save_path = f"pruned_model_{int(pruning_ratio*100)}percent.pth"
    torch.save(model.state_dict(), save_path)
    print(f"Saved pruned model to {save_path}")
    
    return model

def compare_model_sizes(models_dict):
    """Compare and display sizes of multiple models"""
    print("\n=== Model Size Comparison ===")
    print("\nModel Statistics:")
    print("-" * 100)
    print(f"{'Model':20} {'Trainable Params':>15} {'Total Params':>15} {'Memory (MB)':>15}")
    print("-" * 100)
    
    for name, model in models_dict.items():
        analysis = get_model_size(model)
        stats = analysis['model_stats']
        print(f"{name:20} {stats['trainable_parameters']:15,d} {stats['total_parameters']:15,d} {stats['total_memory_mb']:15.2f}")
    
    print("-" * 100)

In [45]:
# 실행 코드
pruning_ratios = [0.3, 0.5, 0.7]
pruned_models = {}

# 각 프루닝 비율에 대해 모델 생성 및 학습
for ratio in pruning_ratios:
    pruned_models[f"Pruned_{int(ratio*100)}%"] = create_and_train_pruned_model(
        ratio, 
        train_loader, 
        val_loader, 
        num_epochs=10
    )

# 원본 모델 추가
original_model = MaskClassifier().to(device)
original_model.load_state_dict(torch.load("mask_classifier.pth"))
pruned_models["Original"] = original_model

# 모델 크기 비교
compare_model_sizes(pruned_models)


Applying 30.0% pruning...

Training 30.0% pruned model...


Epoch 10/10: 100%|██████████| 144/144 [00:58<00:00,  2.44batch/s, loss=0.0340, acc=92.76%]



=== Final Training Results ===
Final Validation Accuracy: 95.93%
Saved pruned model to pruned_model_30percent.pth

Applying 50.0% pruning...

Training 50.0% pruned model...


Epoch 10/10: 100%|██████████| 144/144 [00:41<00:00,  3.45batch/s, loss=0.1014, acc=92.28%]



=== Final Training Results ===
Final Validation Accuracy: 95.93%
Saved pruned model to pruned_model_50percent.pth

Applying 70.0% pruning...

Training 70.0% pruned model...


Epoch 10/10: 100%|██████████| 144/144 [00:32<00:00,  4.40batch/s, loss=0.0726, acc=90.67%]



=== Final Training Results ===
Final Validation Accuracy: 94.70%
Saved pruned model to pruned_model_70percent.pth

=== Model Size Comparison ===

Model Statistics:
----------------------------------------------------------------------------------------------------
Model                Trainable Params    Total Params     Memory (MB)
----------------------------------------------------------------------------------------------------
Pruned_30%                    45,195          45,195            0.17
Pruned_50%                    23,938          23,938            0.09
Pruned_70%                     8,556           8,556            0.03
Original                      93,954          93,954            0.36
----------------------------------------------------------------------------------------------------
